In [196]:
import pandas as pd
import numpy as np 
from numpy import array
import itertools
import keras
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Conv2D,Embedding
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense,LSTM,GRU
from keras.layers import Activation, Dropout,BatchNormalization
from keras import optimizers
from keras import applications
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras import applications 
from keras.utils.np_utils import to_categorical 
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
import math 
import datetime
import time
import tensorflow as tf
from keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
import requests,io,random,re

In [197]:
import seaborn as sns
import sys
import os
import ssl
import random
import cv2 
import numpy as np
ssl._create_default_https_context = ssl._create_unverified_context

In [198]:
text = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')


In [199]:
tx='Please note that we choose to Keras sequential model here since all the layers in the model only have single input and produce single output. In case you want to retrieve and reuse the states from stateful RNN layer, you might want to build your model with Keras functional API or model subclassing. Please check Keras RNN guide for more details.'

In [200]:
text1 = open(text, 'rb').read()
text1=text1.decode(encoding='utf-8')
#text1=tx
print ('Length of text:' ,len(text1))


Length of text: 1115394


In [201]:
text1=text1[:2000]

In [202]:
print(text1[:500])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


In [203]:
num_unique_char=set(text1)
num_unique_char=sorted(num_unique_char)
print(len(num_unique_char))

49


In [204]:
ch_to_ind={c:i for i,c in enumerate(num_unique_char)}
ind_to_ch=np.array(num_unique_char)



In [205]:
for ch in (num_unique_char):
    print(ch ,':', ch_to_ind[ch])


 : 0
  : 1
! : 2
' : 3
, : 4
- : 5
. : 6
: : 7
; : 8
? : 9
A : 10
B : 11
C : 12
F : 13
I : 14
L : 15
M : 16
N : 17
O : 18
R : 19
S : 20
T : 21
V : 22
W : 23
Y : 24
a : 25
b : 26
c : 27
d : 28
e : 29
f : 30
g : 31
h : 32
i : 33
j : 34
k : 35
l : 36
m : 37
n : 38
o : 39
p : 40
r : 41
s : 42
t : 43
u : 44
v : 45
w : 46
y : 47
z : 48


In [206]:
text_as_int=np.array([ch_to_ind[c] for c in text1])

In [207]:

print (text1[:20], text_as_int[:13])


First Citizen:
Befor [13 33 41 42 43  1 12 33 43 33 48 29 38]


In [208]:
max_len=100
num_batch=len(text1)//(max_len+1)



In [209]:
ch_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)


In [210]:
for i in ch_dataset.take(8):
    print(ind_to_ch[i.numpy()])

F
i
r
s
t
 
C
i


In [211]:
seq = ch_dataset.batch(max_len+1, drop_remainder=True)


In [212]:
for i in seq.take(8):
    print(ind_to_ch[i])

['F' 'i' 'r' 's' 't' ' ' 'C' 'i' 't' 'i' 'z' 'e' 'n' ':' '\n' 'B' 'e' 'f'
 'o' 'r' 'e' ' ' 'w' 'e' ' ' 'p' 'r' 'o' 'c' 'e' 'e' 'd' ' ' 'a' 'n' 'y'
 ' ' 'f' 'u' 'r' 't' 'h' 'e' 'r' ',' ' ' 'h' 'e' 'a' 'r' ' ' 'm' 'e' ' '
 's' 'p' 'e' 'a' 'k' '.' '\n' '\n' 'A' 'l' 'l' ':' '\n' 'S' 'p' 'e' 'a'
 'k' ',' ' ' 's' 'p' 'e' 'a' 'k' '.' '\n' '\n' 'F' 'i' 'r' 's' 't' ' ' 'C'
 'i' 't' 'i' 'z' 'e' 'n' ':' '\n' 'Y' 'o' 'u' ' ']
['a' 'r' 'e' ' ' 'a' 'l' 'l' ' ' 'r' 'e' 's' 'o' 'l' 'v' 'e' 'd' ' ' 'r'
 'a' 't' 'h' 'e' 'r' ' ' 't' 'o' ' ' 'd' 'i' 'e' ' ' 't' 'h' 'a' 'n' ' '
 't' 'o' ' ' 'f' 'a' 'm' 'i' 's' 'h' '?' '\n' '\n' 'A' 'l' 'l' ':' '\n'
 'R' 'e' 's' 'o' 'l' 'v' 'e' 'd' '.' ' ' 'r' 'e' 's' 'o' 'l' 'v' 'e' 'd'
 '.' '\n' '\n' 'F' 'i' 'r' 's' 't' ' ' 'C' 'i' 't' 'i' 'z' 'e' 'n' ':'
 '\n' 'F' 'i' 'r' 's' 't' ',' ' ' 'y' 'o' 'u' ' ' 'k']
['n' 'o' 'w' ' ' 'C' 'a' 'i' 'u' 's' ' ' 'M' 'a' 'r' 'c' 'i' 'u' 's' ' '
 'i' 's' ' ' 'c' 'h' 'i' 'e' 'f' ' ' 'e' 'n' 'e' 'm' 'y' ' ' 't' 'o' ' '
 't' 'h' 'e' ' ' 'p

In [213]:
for i in seq.take(8):
    print(''.join(ind_to_ch[i]))

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k
now Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us ki
ll him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be d
one: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citi
zens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but th
e superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we a
re too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particula


In [214]:
for i in seq.take(8):
    print(repr(''.join(ind_to_ch[i])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'
'zens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but th'
'e superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we a'
're too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particula'


In [215]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = seq.map(split_input_target)


In [216]:
for input_example, target_example in  dataset.take(1):
    print ('Input data: ', repr(''.join(ind_to_ch[input_example.numpy()])))
    print ('Target data:', repr(''.join(ind_to_ch[target_example.numpy()])))


Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [217]:
BATCH_SIZE=64
BUFFER_SIZE=1000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [218]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(ind_to_ch[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(ind_to_ch[target_idx])))


Step    0
  input: 13 ('F')
  expected output: 33 ('i')
Step    1
  input: 33 ('i')
  expected output: 41 ('r')
Step    2
  input: 41 ('r')
  expected output: 42 ('s')
Step    3
  input: 42 ('s')
  expected output: 43 ('t')
Step    4
  input: 43 ('t')
  expected output: 1 (' ')


In [219]:
# Length of the vocabulary in chars
vocab_size = len(num_unique_char)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024


In [220]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)])
    return model

In [221]:
model = build_model(
  vocab_size =vocab_size,
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)


In [222]:
rnn.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (64, None, 256)           512000    
_________________________________________________________________
gru_12 (GRU)                 (64, None, 1024)          3935232   
_________________________________________________________________
dense_8 (Dense)              (64, None, 49)            50225     
Total params: 4,497,457
Trainable params: 4,497,457
Non-trainable params: 0
_________________________________________________________________


In [229]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batchc_predictions = rnn(input_example_batch)
    print(example_batchc_predictions.shape, "# (batch_size, sequence_length, vocab_size)")


In [230]:
rnn.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (64, None, 256)           512000    
_________________________________________________________________
gru_12 (GRU)                 (64, None, 1024)          3935232   
_________________________________________________________________
dense_8 (Dense)              (64, None, 49)            50225     
Total params: 4,497,457
Trainable params: 4,497,457
Non-trainable params: 0
_________________________________________________________________


In [231]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()


InvalidArgumentError: logits should be a matrix, got shape []
	 [[{{node Multinomial}}]] [Op:Multinomial]

In [193]:
rnn.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [194]:
rnn.fit(dataset,epochs=10)

AttributeError: 'BatchDataset' object has no attribute 'ndim'